In [1]:
# Imports

import pandas as pd
from pandas import Series,DataFrame
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

Load titanic train and test csv files

In [3]:
X_train = pd.read_csv("./train.csv")
y_train = X_train.pop("Survived")

X_test_orig = pd.read_csv("./test.csv")

In [4]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [6]:

data = X_train.append(X_test_orig, ignore_index=True)

In [7]:
data['Title'] = data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
data.Title.value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Ms                2
Major             2
Mlle              2
the Countess      1
Jonkheer          1
Sir               1
Mme               1
Dona              1
Capt              1
Lady              1
Don               1
Name: Title, dtype: int64

In [8]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
data.Title = data.Title.map(normalized_titles)
data.Title.value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64

In [9]:
# group by Sex, Pclass, and Title
grouped = data.groupby(['Sex','Pclass', 'Title'])

In [10]:
grouped.Age.median()

Sex     Pclass  Title  
female  1       Miss       30.0
                Mrs        45.0
                Officer    49.0
                Royalty    39.0
        2       Miss       20.0
                Mrs        30.0
        3       Miss       18.0
                Mrs        31.0
male    1       Master      6.0
                Mr         41.5
                Officer    52.0
                Royalty    40.0
        2       Master      2.0
                Mr         30.0
                Officer    41.5
        3       Master      6.0
                Mr         26.0
Name: Age, dtype: float64

In [11]:
data.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))
data.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
Title             0
dtype: int64

In [12]:
data['CatAge'] = pd.qcut(data.Age, q=5, labels=False )

In [13]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,CatAge
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,3
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,3
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,3


In [14]:
#Add a new family size feature 
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1
data.describe()


,PassengerId,Pclass,Age,SibSp,Parch,Fare,CatAge,Family_Size
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1308.000000,1309.000000,1309.000000
mean,655.000000,2.294882,29.278205,0.498854,0.385027,33.295479,1.853323,1.883881
std,378.020061,0.837836,13.446690,1.041658,0.865560,51.758668,1.451520,1.583639
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800,1.000000,1.000000
50%,655.000000,3.000000,26.000000,0.000000,0.000000,14.454200,1.000000,1.000000
75%,982.000000,3.000000,36.500000,1.000000,0.000000,31.275000,3.000000,2.000000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200,4.000000,11.000000


In [15]:
data["Fare"].fillna(data.Fare.mean(), inplace=True)

most_embarked = data.Embarked.value_counts().index[0]
data["Embarked"].fillna(most_embarked, inplace = True)
data.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
Title             0
CatAge            0
Family_Size       0
dtype: int64

In [16]:
#drop columns won't be useful in analysis and prediction
data = data.drop(['PassengerId','Name','Ticket','Cabin','Age','SibSp','Parch'], axis=1)

In [17]:
data = pd.get_dummies(data, columns=['Sex'], prefix = ['Sex'])


In [18]:
data = pd.get_dummies(data, columns=['Embarked'], prefix = ['Embarked'])


In [19]:
data = pd.get_dummies(data, columns=['Title'], prefix = ['Title'])

In [20]:
data.head()

,Pclass,Fare,CatAge,Family_Size,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,3,7.2500,1,2,0,1,0,0,1,0,0,1,0,0,0
1,1,71.2833,3,2,1,0,1,0,0,0,0,0,1,0,0
2,3,7.9250,1,1,1,0,0,0,1,0,1,0,0,0,0
3,1,53.1000,3,2,1,0,0,0,1,0,0,0,1,0,0
4,3,8.0500,3,1,0,1,0,0,1,0,0,1,0,0,0


In [21]:
X_train = data.iloc[:891]
X_test = data.iloc[891:]

# SVC Classifier

In [44]:
from sklearn.svm import SVC
svc_params = [
   { 'C': [0.001, 0.01, 0.1, 1, 5, 10], 
     'gamma' : [0.001, 0.01, 0.1],
   },
]


clf = SVC(svc_params)
clf = GridSearchCV(clf, svc_params, cv=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Best score: {}".format(clf.best_score_))
print("Optimal params: {}".format(clf.best_estimator_))


Best score: 0.8148148148148148
Optimal params: SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [45]:
submission = pd.DataFrame({
        "PassengerId": X_test_orig["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('titanic_SVC.csv', index=False)

# Ramdom Forest Classifier

In [24]:
forest_params = [dict(
   max_depth = [n for n in range(9, 14)],
   min_samples_split = [n for n in range(4, 11)],
   min_samples_leaf = [n for n in range(2, 5)],
   n_estimators = [n for n in range(10, 60, 10)],
)]


model = RandomForestClassifier(forest_params)
model = GridSearchCV(model, forest_params, cv=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [25]:
submission = pd.DataFrame({
        "PassengerId": X_test_orig["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('titanic_RFC.csv', index=False)

In [26]:
X_train_small = data.iloc[:800]
X_test_small = data.iloc[800:891]
y_train_small = y_train.iloc[:800]
y_test_small = y_train.iloc[800:891]

# Decision Tree Classifier

In [27]:
from sklearn.tree import DecisionTreeClassifier


In [28]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train_small, y_train_small)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [29]:
cross_val_score(dt, X_train_small, y_train_small, cv=5, scoring="accuracy")

array([0.76397516, 0.78881988, 0.80625   , 0.79245283, 0.80503145])

In [30]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.79437564499484

# Using GridSearch with Decision Tree

In [31]:
dt_params = {
    'max_depth' : [1,2,4,8,16,32,64],
    'min_samples_leaf' : [1,2,3,4,5,6,7]
}

In [32]:
dt1 = GridSearchCV(dt, dt_params, cv=5)

In [33]:
dt1.fit(X_train_small, y_train_small)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 4, 8, 16, 32, 64], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [34]:
cross_val_score(dt1.best_estimator_, X_train_small, y_train_small, cv=5, scoring="accuracy")

array([0.80745342, 0.80745342, 0.86875   , 0.8490566 , 0.80503145])

In [35]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt1.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.8062435500515996

# Random Forest Classifier to Find the Importance of each Feature.


In [36]:
rnd_clf = RandomForestClassifier()
rnd_clf.fit(X_train_small, y_train_small)
feat_labels = X_train_small.columns
importances = rnd_clf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(15):
    print("%2d) %-*s %f" % (f + 1, 20, feat_labels[indices[f]], importances[indices[f]]))

 1) Fare                 0.337778
 2) Sex_female           0.192896
 3) CatAge               0.102743
 4) Family_Size          0.092012
 5) Pclass               0.090600
 6) Sex_male             0.051982
 7) Title_Miss           0.042246
 8) Title_Mr             0.024477
 9) Embarked_S           0.015667
10) Embarked_C           0.014736
11) Embarked_Q           0.012266
12) Title_Master         0.011160
13) Title_Mrs            0.006580
14) Title_Officer        0.002513
15) Title_Royalty        0.002345


# Voting Classifier - Three Non-Ensemble Classifier

In [37]:
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=1)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = SVC()
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1.fit(X_train_small, y_train_small)
cross_val_score(eclf1, X_train_small, y_train_small, cv=5, scoring="accuracy")

/Users/sharviljani/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/sharviljani/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/sharviljani/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/sharviljani/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

array([0.79503106, 0.80124224, 0.8375    , 0.80503145, 0.83018868])

# Adaboost Classfier and GridSearch CV

In [38]:
dt1 = GridSearchCV(dt, dt_params, cv=5)

In [39]:
from sklearn.ensemble import AdaBoostClassifier
bdt_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5, min_samples_leaf=7),
                             algorithm=DecisionTreeClassifier,
                             learning_rate=0.0003,
                             n_estimators=7)


In [40]:
dt1.fit(X_train_small, y_train_small)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 4, 8, 16, 32, 64], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
cross_val_score(dt1.best_estimator_, X_train_small, y_train_small, cv=5, scoring="accuracy")

array([0.80745342, 0.80745342, 0.86875   , 0.8490566 , 0.80503145])

In [42]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt1.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.8062435500515996